In [3]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

def create_database(db_name="thematic_yearly_survey", user="postgres", password="PUT OWN PASSWORD", host="localhost", port="5432"):
    conn = psycopg2.connect(
        dbname="postgres",  # connect to default db
        user=user,
        password=password,
        host=host,
        port=port
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()

    cur.execute(f"SELECT 1 FROM pg_database WHERE datname='{db_name}'")
    exists = cur.fetchone()
    if not exists:
        cur.execute(f"CREATE DATABASE {db_name}")
        print(f" Database '{db_name}' created.")
    else:
        print(" Database already exists.")

    cur.close()
    conn.close()





In [4]:
import os
import pandas as pd
import math
import psycopg2
from psycopg2.extras import Json
from textblob import TextBlob
import re

# --- Configuration ---
FOLDER = "."
comment_keywords = ["comment", "feedback", "remark"]
binary_options = ["yes", "no", "male", "female"]

def clean_nan(obj):
    if isinstance(obj, dict):
        return {k: clean_nan(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_nan(v) for v in obj]
    elif isinstance(obj, float) and (math.isnan(obj) or pd.isna(obj)):
        return None
    elif isinstance(obj, (pd._libs.missing.NAType, type(pd.NA))):
        return None
    return obj

def categorize_sentiment(text):
    if not isinstance(text, str) or not text.strip():
        return {"label": None, "polarity": None}
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        label = "positive"
    elif polarity < -0.1:
        label = "negative"
    else:
        label = "neutral"
    return {"label": label, "polarity": round(polarity, 3)}

def is_comment(col, series):
    if any(key in col.lower() for key in comment_keywords):
        return True
    if pd.api.types.is_object_dtype(series):
        non_null_series = series.dropna().astype(str)
        sample = non_null_series.sample(n=min(20, len(non_null_series)), random_state=1)
        avg_len = sample.map(len).mean() if not sample.empty else 0
        long_texts = sample[sample.map(len) > 30]
        return avg_len > 20 and len(long_texts) > 0
    return False

def is_quantitative(series):
    return pd.api.types.is_numeric_dtype(series) and series.dropna().between(1, 10).all()

def is_demographic(series):
    if series.dtype == object:
        unique_vals = series.dropna().astype(str).str.lower().unique().tolist()
        return len(unique_vals) <= 10 or all(val in binary_options for val in unique_vals)
    return False

def ingest_excel_to_postgres(db_name="thematic_yearly_survey", user="postgres", password="PUT OWN PASSWORD", host="localhost", port="5432"):
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host, port=port)
    conn.autocommit = True
    cur = conn.cursor()

    for file in os.listdir(FOLDER):
        if file.endswith(".xlsx"):
            print(f"\n Processing file: {file}")
            basename = os.path.splitext(file)[0]
            match = re.search(r"\d{4}", basename)
            year = match.group(0) if match else "unknown"
            if year == "unknown":
                raise ValueError(f"No valid 4-digit year found in filename: {file}")
            table_name = f"theme_blocks_{year}".lower()

            df = pd.read_excel(os.path.join(FOLDER, file))
            df.columns = df.columns.str.strip()

            col_types = {}
            for col in df.columns:
                if is_comment(col, df[col]):
                    col_types[col] = "comment"
                elif is_quantitative(df[col]):
                    col_types[col] = "quantitative"
                elif is_demographic(df[col]):
                    col_types[col] = "demographic"
                else:
                    col_types[col] = "unknown"

            cur.execute(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    id SERIAL PRIMARY KEY,
                    respondent_index INT,
                    year TEXT,
                    demographics JSONB,
                    questions JSONB,
                    comment TEXT,
                    sentiment JSONB
                );
            """)

            for idx, row in df.iterrows():
                respondent_index = int(idx) + 1
                demographics = {
                    col: str(row[col]).strip()
                    for col in df.columns
                    if col_types[col] == "demographic" and pd.notna(row[col]) and str(row[col]).strip() != ""
                }

                theme_blocks = []
                current_questions = {}

                for col in df.columns:
                    value = row[col]
                    col_type = col_types[col]

                    if col_type == "quantitative":
                        current_questions[col] = value
                    elif col_type == "comment":
                        sentiment_info = categorize_sentiment(value)
                        theme_blocks.append({
                            "respondent_index": respondent_index,
                            "year": year,
                            "demographics": clean_nan(dict(demographics)),
                            "questions": clean_nan(dict(current_questions)),
                            "comment": clean_nan(value),
                            "sentiment": sentiment_info
                        })
                        current_questions = {}

                if current_questions:
                    theme_blocks.append({
                        "respondent_index": respondent_index,
                        "year": year,
                        "demographics": clean_nan(dict(demographics)),
                        "questions": clean_nan(current_questions),
                        "comment": None,
                        "sentiment": {"label": None, "polarity": None}
                    })

                for block in theme_blocks:
                    cur.execute(f"""
                        INSERT INTO {table_name} (
                            respondent_index, year, demographics, questions, comment, sentiment
                        ) VALUES (%s, %s, %s, %s, %s, %s)
                    """, (
                        block["respondent_index"],
                        block["year"],
                        Json(block["demographics"]),
                        Json(block["questions"]),
                        block["comment"],
                        Json(block["sentiment"])
                    ))

            print(f" Inserted data into table: {table_name}")

    cur.close()
    conn.close()

In [5]:
import psycopg2
import pandas as pd
import re

def load_all_dataframes(db_name="thematic_yearly_survey", user="postgres", password="PUTOWNPASSWORD", host="localhost", port="5432"):
    conn = psycopg2.connect(
        dbname=db_name,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = conn.cursor()

    cur.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        AND table_name LIKE 'theme_blocks_%';
    """)
    table_names = [row[0] for row in cur.fetchall()]

    loaded_dataframes = {}

    for table in table_names:
        match = re.search(r'theme_blocks_(\d{4})', table)
        if match:
            year = match.group(1)
            df = pd.read_sql_query(f"SELECT * FROM {table};", conn)
            var_name = f"dataframe_{year}"
            loaded_dataframes[var_name] = df
            print(f" Loaded '{var_name}' with {len(df)} rows")

    cur.close()
    conn.close()
    loaded_dataframes['__names__'] = list(loaded_dataframes.keys())
    return loaded_dataframes


# flatten data for sentiment polarity

In [6]:
def extract_sentiment_fields(df, sentiment_col='sentiment', fields=None):

    if fields is None:
        fields = ['label', 'polarity']

    for field in fields:
        df[f"{sentiment_col}_{field}"] = df[sentiment_col].apply(
            lambda x: x.get(field) if isinstance(x, dict) else None
        )

    return df


###### now  this is to create proper sentiment and polarity columns

In [7]:
def clean_and_rename_sentiment_fields(df_dict):
    updated_count = 0

    for name, df in df_dict.items():
        modified = False

        # Drop 'sentiment' column if it exists
        if 'sentiment' in df.columns:
            df.drop(columns=['sentiment'], inplace=True)
            modified = True

        # Rename 'sentiment_label' to 'sentiment' if not already renamed
        if 'sentiment_label' in df.columns and 'sentiment' not in df.columns:
            df.rename(columns={'sentiment_label': 'sentiment'}, inplace=True)
            modified = True

        if modified:
            updated_count += 1
            print(f" Cleaned {name}")

    print(f" Total cleaned DataFrames: {updated_count}")
    return updated_count
#

In [8]:
# open api key

In [30]:

import openai


# Set API key
client = openai.OpenAI(api_key="sk-")  #



###### extraction of demographic keys

In [10]:
def extract_unique_demographic_keys(df_dict, column_name="demographics"):

    def extract_keys(d):
        return list(d.keys()) if isinstance(d, dict) else []

    all_keys = []

    for name, df in df_dict.items():
        if column_name in df.columns:
            nested_keys = df[column_name].apply(extract_keys)
            flat_keys = [key for sublist in nested_keys for key in sublist]
            all_keys.extend(flat_keys)
            print(f" Extracted keys from {name}")

    unique_keys = sorted(set(all_keys))
    print(f" demographic_keys = {unique_keys}")
    return unique_keys


###### demographic prompt

In [11]:
def generate_demographic_mapping_prompt(keys):

    prompt = (
    "You are a schema normalization assistant. Given a list of demographic survey field names:\n\n"
    + "\n".join(f"- {key}" for key in keys) +
    "\n\nYour task is to:\n"
    "1. Normalize each key into a canonical, consistent label.\n"
    "2. Avoid spelling-based confusion (e.g., 'Carer' ≠ 'Career').\n"
    "3. Where multiple keys refer to the same underlying concept (e.g., 'Ethnicity' and 'What is your ethnic background?'), unify them to the same canonical form.\n"
    "4. If a field is ambiguous like 'Are you?', treat it as 'Gender' only if options suggest that.\n\n"
    "Return a Python dictionary in this format:\n"
    "{'Original Key': 'CleanedKey', ...}"
      )
    return prompt


###### get key mapping from llm

In [12]:
# Pricing per 1K tokens (as of 2025-08, adjust if OpenAI updates)
PRICING = {
    "gpt-4":         {"input": 0.03,  "output": 0.06},  # $ per 1K tokens
    "gpt-4-turbo":   {"input": 0.01,  "output": 0.03},  # Turbo is cheaper
    "gpt-3.5-turbo": {"input": 0.001, "output": 0.002}
}

# Track totals for each model
token_usage_summary = {
    "gpt-4":         {"input_tokens": 0, "output_tokens": 0, "cost": 0.0},
    "gpt-4-turbo":   {"input_tokens": 0, "output_tokens": 0, "cost": 0.0},
    "gpt-3.5-turbo": {"input_tokens": 0, "output_tokens": 0, "cost": 0.0}
}

def get_cleaned_key_mapping(prompt, model_name="gpt-4", temperature=0.4):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You help with standardizing dataset keys."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )

    # --- Record usage ---
    usage = response.usage
    input_tokens = usage.prompt_tokens
    output_tokens = usage.completion_tokens

    # Compute cost
    price = PRICING.get(model_name, {"input": 0, "output": 0})
    cost = (input_tokens / 1000 * price["input"]) + (output_tokens / 1000 * price["output"])

    # Update totals
    if model_name not in token_usage_summary:
        token_usage_summary[model_name] = {"input_tokens": 0, "output_tokens": 0, "cost": 0.0}

    token_usage_summary[model_name]["input_tokens"]  += input_tokens
    token_usage_summary[model_name]["output_tokens"] += output_tokens
    token_usage_summary[model_name]["cost"]          += cost

    print(f"[{model_name}] Input tokens: {input_tokens}, Output tokens: {output_tokens}, Cost: ${cost:.6f}")

    return response.choices[0].message.content

def print_usage_summary():
    print("\n=== Token Usage Summary ===")
    for model, stats in token_usage_summary.items():
        total_tokens = stats["input_tokens"] + stats["output_tokens"]
        print(f"{model}: {total_tokens} tokens "
              f"(input={stats['input_tokens']}, output={stats['output_tokens']}), "
              f"Total cost=${stats['cost']:.6f}")


###### clean the extra output from llm and ask user to chose the mapping ( optional )

In [13]:
import ast
import re

# --- Core Parser ---
def extract_first_dict_from_response(text, verbose=True):
    """
    Extracts the first dictionary-like structure from an LLM response string.
    """
    match = re.search(r"\{.*?\}", text, re.DOTALL)
    if match:
        dict_text = match.group()
        try:
            result = ast.literal_eval(dict_text)
            if verbose:
                print(" Successfully extracted dictionary.")
            return result
        except Exception as e:
            if verbose:
                print(f" Failed to parse dictionary: {e}")
            return None
    else:
        if verbose:
            print(" No dictionary block found in text.")
        return None

# --- Selector ---
def choose_llm_mapping(mapping_responses):
    """
    """
    print("\n Available model responses:")
    options = list(mapping_responses.keys())
    for i, model in enumerate(options, start=1):
        print(f"{i}. {model.upper()}")

    while True:
        try:
            choice = int(input("Select a model response to extract mapping from (1 or 2): "))
            if 1 <= choice <= len(options):
                selected_model = options[choice - 1]
                print(f"\n Extracting mapping from {selected_model.upper()} response...\n")
                mapping_dict = extract_first_dict_from_response(mapping_responses[selected_model])
                if mapping_dict is not None:
                    print(" Extracted Mapping:\n")
                    for k, v in mapping_dict.items():
                        print(f"  {repr(k)}: {repr(v)}")
                return mapping_dict
            else:
                print(" Invalid choice. Please enter 1 or 2.")
        except ValueError:
            print(" Please enter a number.")


###### now apply mappings to data frames

In [14]:

import re
import pandas as pd

#  Key normalization function
def normalize_demographic_keys(demo_dict, mapping):
    if isinstance(demo_dict, dict):
        return {
            mapping.get(k.strip(), k.strip()): v
            for k, v in demo_dict.items()
        }
    return demo_dict

#  Apply demographic normalization to all unpacked_dfs and sync to globals
def normalize_all_demographics(unpacked_dfs, mapping):
    for name, df in unpacked_dfs.items():
        if isinstance(df, pd.DataFrame) and "demographics" in df.columns:
            df_normalized = df.copy()
            df_normalized["demographics"] = df_normalized["demographics"].apply(
                lambda d: normalize_demographic_keys(d, mapping)
            )
            unpacked_dfs[name] = df_normalized
            globals()[name] = df_normalized  # sync with global variable too
            print(f" Normalized demographics for {name}")



#  Optional preview
# unpacked_dfs["df_2023"]["demographics"].dropna().head()


normalization of questions, full process

In [17]:
import re
import pandas as pd

def extract_question_keys(row):
    """Extract keys from a dictionary in the 'questions' column."""
    if isinstance(row, dict):
        return list(row.keys())
    return []

def get_question_keys_from_dataframe(df: pd.DataFrame) -> list:
    """Extract all question keys from the 'questions' column of a DataFrame."""
    if "questions" not in df.columns:
        return []
    keys_series = df["questions"].apply(extract_question_keys)
    return [key for sublist in keys_series for key in sublist]

def collect_question_keys_from_unpacked_dfs(unpacked_dfs: dict) -> list:
    """Scan unpacked DataFrames and extract question keys."""
    all_keys = []
    for df_name, df in unpacked_dfs.items():
        if isinstance(df, pd.DataFrame):
            if "questions" in df.columns:
                keys = get_question_keys_from_dataframe(df)
                all_keys.extend(keys)
                print(f" Processed {df_name}")
    return sorted(set(all_keys))


from typing import Dict, List

# ---------- Pricing (per 1K tokens) – update to your rates ----------
PRICING = {
    "gpt-4":         {"input": 0.03,  "output": 0.06},
    "gpt-4-turbo":   {"input": 0.01,  "output": 0.03},    # example
    "gpt-3.5-turbo": {"input": 0.001, "output": 0.002},
}

# ---------- Running totals ----------
token_usage_summary = {
    m: {"input_tokens": 0, "output_tokens": 0, "cost": 0.0}
    for m in PRICING.keys()
}

def _record_usage(model_name: str, usage) -> None:
    """Accumulate tokens & cost into token_usage_summary for a single response."""
    if usage is None:
        return
    input_tokens = getattr(usage, "prompt_tokens", None) or getattr(usage, "prompt_tokens", 0)
    output_tokens = getattr(usage, "completion_tokens", None) or getattr(usage, "completion_tokens", 0)

    price = PRICING.get(model_name, {"input": 0.0, "output": 0.0})
    cost = (input_tokens / 1000.0) * price["input"] + (output_tokens / 1000.0) * price["output"]

    bucket = token_usage_summary.setdefault(model_name, {"input_tokens": 0, "output_tokens": 0, "cost": 0.0})
    bucket["input_tokens"]  += input_tokens
    bucket["output_tokens"] += output_tokens
    bucket["cost"]          += cost

    print(f"[{model_name}] input={input_tokens}, output={output_tokens}, cost=${cost:.6f}")


# ---------- Your prompt builder (unchanged) ----------
def build_question_normalization_prompt(question_keys: List[str]) -> str:
    formatted_keys = "\n".join(f"- {key}" for key in question_keys)
    example_mapping = """{
    'Public transport': 'Public Transport',
    'Public & Transport': 'Public Transport',
    'Sense of Community 1': 'Sense of Community 1',
    'Sense of Community 2': 'Sense of Community 2'
}"""
    return (
        "You are a data normalization assistant. Below is a list of survey question keys:\n\n"
        f"{formatted_keys}\n\n"
        "Your task is to return a Python dictionary that groups semantically similar phrases under one consistent label.\n"
        "However, preserve numeric suffixes where they appear to represent different items (e.g., 'Sense of Community 1', '... 2', etc.).\n"
        "Do not merge them into one label like 'Sense of Community'.\n"
        "Only group entries when they are true paraphrases of the same concept, not when they are distinct numbered items.\n\n"
        "Return the result in valid Python dictionary format. For example:\n"
        f"{example_mapping}\n\n"
        "Return only the dictionary — no explanations."
    )


# ---------- Updated model call with usage & cost tracking ----------
def get_question_mapping_from_model(model_name: str, prompt: str) -> Dict[str, str]:
    """Calls an OpenAI model to get a question-key normalization mapping, and records token usage & cost."""
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You help clean and normalize field names in datasets."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0
    )

    # record usage & cost
    _record_usage(model_name, getattr(response, "usage", None))

    content = response.choices[0].message.content.strip()
    try:
        mapping = ast.literal_eval(content)
        assert isinstance(mapping, dict)
    except Exception as e:
        print(f"Failed to parse response from {model_name}: {e}")
        mapping = {}
    return mapping

def normalize_question_keys_with_models(question_keys: List[str]) -> Dict[str, Dict[str, str]]:
    """Builds the prompt and queries both GPT-4 and GPT-3.5 for normalization mappings."""
    prompt = build_question_normalization_prompt(question_keys)

    gpt4_mapping = get_question_mapping_from_model("gpt-4", prompt)
    print("\n GPT-4 Question Normalization Mapping:\n", gpt4_mapping)

    gpt35_mapping = get_question_mapping_from_model("gpt-3.5-turbo", prompt)
    print("\n GPT-3.5 Question Normalization Mapping:\n", gpt35_mapping)

    gpt4_turbo_mapping = get_question_mapping_from_model("gpt-4-turbo", prompt)
    print("\n GPT-4 turbo Question Normalization Mapping:\n", gpt4_turbo_mapping)
    print_usage_summary()
    return {
        "gpt-4": gpt4_mapping,
        "gpt-3.5-turbo": gpt35_mapping,
        "gpt-4-turbo": gpt4_turbo_mapping

    }

def choose_question_key_mapping(mappings: dict) -> dict:
    """Display both GPT mappings and let the user choose one."""
    print("\n GPT-4 Normalization Mapping:\n")
    for k, v in mappings.get("gpt-4", {}).items():
        print(f"{k!r}: {v!r}")

    print("\n GPT-3.5 Normalization Mapping:\n")
    for k, v in mappings.get("gpt-3.5-turbo", {}).items():
        print(f"{k!r}: {v!r}")

    # Prompt user for selection
    print("\n Choose the mapping you want to apply:")
    print("1 → GPT-3.5")
    print("2 → GPT-4")

    while True:
        choice = input("Enter 1 or 2: ").strip()
        if choice == "1":
            print(" Using GPT-3.5 mapping.")
            return mappings["gpt-3.5-turbo"]
        elif choice == "2":
            print(" Using GPT-4 mapping.")
            return mappings["gpt-4"]
        else:
            print("️ Invalid input. Please enter 1 or 2.")

import ast
import pandas as pd
from typing import Dict

def parse_question_mapping(mapping_str: str) -> Dict[str, str]:
    """Safely parse a string containing a Python dictionary."""
    try:
        mapping = ast.literal_eval(mapping_str)
        assert isinstance(mapping, dict)
        return mapping
    except Exception as e:
        print(f" Failed to parse mapping string: {e}")
        return {}

def normalize_question_dict(q_dict: dict, mapping: dict) -> dict:
    """Normalize the keys in a single question dictionary using the mapping."""
    if isinstance(q_dict, dict):
        return {
            mapping.get(k.strip(), k.strip()): v
            for k, v in q_dict.items()
        }
    return q_dict

def apply_mapping_to_unpacked_dfs(unpacked_dfs: dict, mapping: dict) -> dict:
    """
    Apply a question key normalization mapping to all DataFrames in unpacked_dfs.
    Returns a new dictionary of normalized DataFrames.
    """
    normalized_dfs = {}
    for name, df in unpacked_dfs.items():
        if isinstance(df, pd.DataFrame) and "questions" in df.columns:
            df_normalized = df.copy()
            df_normalized["questions"] = df["questions"].apply(lambda q: normalize_question_dict(q, mapping))
            normalized_dfs[name] = df_normalized
            print(f" Normalized questions for {name}")
        else:
            normalized_dfs[name] = df  # preserve untouched if no 'questions' column
    return normalized_dfs


explode questions before macro theme

In [18]:
def explode_questions_into_named_ratings(unpacked_dfs: dict, source_col: str = "questions") -> None:
    """
    Explode 'questions' dict column in each DataFrame in unpacked_dfs.
    Adds 'namedquestion' and 'rating' columns.
    Updates unpacked_dfs in-place.
    """
    for name, df in unpacked_dfs.items():
        if isinstance(df, pd.DataFrame) and source_col in df.columns:
            records = []

            for _, row in df.iterrows():
                q_dict = row.get(source_col)
                if isinstance(q_dict, dict):
                    for k, v in q_dict.items():
                        new_row = row.drop(labels=[source_col]).to_dict()
                        new_row["question_label"] = k
                        new_row["rating_label"] = v if isinstance(v, (int, float)) else None
                        records.append(new_row)
                else:
                    new_row = row.to_dict()
                    new_row["question_label"] = None
                    new_row["rating_label"] = None
                    records.append(new_row)

            unpacked_dfs[name] = pd.DataFrame(records)
            print(f"Exploded and updated: {name}")


macro theme

In [19]:
#  Macro theme mapping (unchanged)
MACRO_THEME_MAP = {
    "Active Travel": "MOVEMENT",
    "Public Transport": "MOVEMENT",
    "Traffic & Parking": "MOVEMENT",
    "Moving Around": "MOVEMENT",

    "Streets & Spaces": "SPACE",
    "Natural Space": "SPACE",
    "Play & Recreation": "SPACE",

    "Facilities & Amenities": "RESOURCES",
    "Work & Local Economy": "RESOURCES",
    "Housing & Community": "RESOURCES",
    "Social Interactions": "RESOURCES",

    "Identity & Belonging": "CIVIC",
    "Feeling Safe": "CIVIC",
    "Sense of Community": "CIVIC",

    "Care & Maintenance": "STEWARDSHIP",
    "Care & Maintenance Are buildings and spaces well cared for?": "STEWARDSHIP",
    "Influence & Control": "STEWARDSHIP"
}


#  Label assigner for namedquestion column
def assign_macro_label_from_namedquestion(named_q: str, mapping: dict) -> str:
    if isinstance(named_q, str):
        return mapping.get(named_q.strip(), None)
    return None

#  Apply macro_label using namedquestion column across all unpacked_dfs
def apply_macro_labels_from_namedquestion(unpacked_dfs: dict, macro_mapping: dict) -> None:
    for name, df in unpacked_dfs.items():
        if isinstance(df, pd.DataFrame) and "question_label" in df.columns:
            df_labeled = df.copy()
            df_labeled["macro_label"] = df_labeled["question_label"].apply(
                lambda x: assign_macro_label_from_namedquestion(x, macro_mapping)
            )
            unpacked_dfs[name] = df_labeled
            globals()[name] = df_labeled  # Sync back
            print(f" Added macro_label from question_label to {name}")


flatten demographics

In [23]:
def flatten_demographics_columns(dataframes: dict) -> dict:
    """
    Flatten the 'demographics' column (dicts) into top-level columns for all DataFrames.
    Returns a new dictionary with keys as 'original_name_flat'.
    Also updates globals for direct use in notebook.
    """
    flattened = {}

    for name, df in dataframes.items():
        if isinstance(df, pd.DataFrame) and "demographics" in df.columns:
            df_flat = pd.concat(
                [df.drop(columns=["demographics"]),
                 df["demographics"].apply(pd.Series)],
                axis=1
            )
            flat_name = f"{name}_flat"
            flattened[flat_name] = df_flat
            globals()[flat_name] = df_flat  # store for interactive use
            print(f" Flattened demographics in {name} → stored as {flat_name}")

    return flattened



data types rectification

In [24]:
import pandas as pd

def infer_and_assign_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Infers and assigns optimal data types for each column in a DataFrame.
    Converts numerics, datetimes, booleans, and low-cardinality strings to categories.
    """
    for col in df.columns:
        inferred = pd.api.types.infer_dtype(df[col], skipna=True)

        if inferred in ['integer', 'floating', 'mixed-integer-float']:
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except Exception:
                pass
        elif inferred in ['datetime', 'datetime64']:
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
            except Exception:
                pass
        elif inferred == 'boolean':
            df[col] = df[col].astype('boolean')
        elif inferred in ['string', 'unicode']:
            unique_vals = df[col].dropna().unique()
            if len(unique_vals) < len(df) * 0.1:  # treat low-cardinality strings as categories
                df[col] = df[col].astype('category')

    return df.convert_dtypes()

def apply_type_inference_to_unpacked_dfs(unpacked_dfs: dict) -> dict:
    """
    Applies dtype inference and optimization to all DataFrames in unpacked_dfs.
    Updates the original unpacked_dfs dictionary in-place and returns it.
    """
    for name, df in unpacked_dfs.items():
        if isinstance(df, pd.DataFrame):
            inferred_df = infer_and_assign_dtypes(df)
            unpacked_dfs[name] = inferred_df
            print(f" Inferred dtypes for {name}")
    return unpacked_dfs


###### postgres ingestion final

In [ ]:
import time
from sqlalchemy import create_engine

def insert_all_flat_dataframes(unpacked_dfs, db_name, user, password, host, port):
    # Create PostgreSQL connection string
    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}")

    for idx, (df_name, df) in enumerate(unpacked_dfs.items(), start=1):
        table_name = f"{df_name}_{idx}"  # e.g., survey_1, survey_2
        print(f"Inserting DataFrame into table: {table_name}")

        # Write DataFrame to PostgreSQL
        df.to_sql(table_name, engine, if_exists='replace', index=False)

    print(" All DataFrames inserted successfully!")


Pipeline begins



main

In [25]:

create_database()
import time

start = time.time()
ingest_excel_to_postgres() # or ingest_excel_to_mongodb()
print("Ingestion Time:", time.time() - start, "seconds")
start = time.time()
start = time.time()
dfs = load_all_dataframes()
print("loading Time:", time.time() - start, "seconds")


 Database already exists.

 Processing file: 2016_Our_Place_Results_Redacted.xlsx
 Inserted data into table: theme_blocks_2016

 Processing file: Place Survey 2018 Data_Redacted.xlsx
 Inserted data into table: theme_blocks_2018

 Processing file: Place Survey 2020 Data_Redacted.xlsx
 Inserted data into table: theme_blocks_2020

 Processing file: SurveyResults-Our_Place_2023_ORIGINAL (1).xlsx
 Inserted data into table: theme_blocks_2023
Ingestion Time: 18.033456325531006 seconds


C:\Users\ltopuser\AppData\Local\Temp\ipykernel_17432\3697741667.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table};", conn)


 Loaded 'dataframe_2023' with 6340 rows
 Loaded 'dataframe_2020' with 6015 rows
 Loaded 'dataframe_2016' with 13695 rows
 Loaded 'dataframe_2018' with 3735 rows
loading Time: 0.9067165851593018 seconds


In [27]:


#  Run the function

dfs = load_all_dataframes()

unpacked_dfs = {}

for key, df in dfs.items():
    if key.startswith("dataframe_"):
        year = key.split("_")[1]
        var_name = f"df_{year}"
        locals()[var_name] = df
        unpacked_dfs[var_name] = df  #  Add this
        print(f"Created variable {var_name} with {len(df)} rows")


for name, df in unpacked_dfs.items():
    extract_sentiment_fields(df)
    print(f" Updated {name}")

# Call the function and pass in your unpacked_dfs dictionary
clean_and_rename_sentiment_fields(unpacked_dfs)


unique_demographic_keys = extract_unique_demographic_keys(unpacked_dfs)





C:\Users\ltopuser\AppData\Local\Temp\ipykernel_17432\3697741667.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table};", conn)


 Loaded 'dataframe_2023' with 6340 rows
 Loaded 'dataframe_2020' with 6015 rows
 Loaded 'dataframe_2016' with 13695 rows
 Loaded 'dataframe_2018' with 3735 rows
Created variable df_2023 with 6340 rows
Created variable df_2020 with 6015 rows
Created variable df_2016 with 13695 rows
Created variable df_2018 with 3735 rows
 Updated df_2023
 Updated df_2020
 Updated df_2016
 Updated df_2018
 Cleaned df_2023
 Cleaned df_2020
 Cleaned df_2016
 Cleaned df_2018
 Total cleaned DataFrames: 4
 Extracted keys from df_2023
 Extracted keys from df_2020
 Extracted keys from df_2016
 Extracted keys from df_2018
 demographic_keys = ['Age', 'Are you?', 'Care for a family member /friend?', 'Carer', 'Disabled', 'Do you care for a family member/friend because of their illness or disability?', 'Do you care for a family member/friend?', 'Do you consider yourself disabled?', 'Do you consider yourself to be disabled?', 'Do you have a hearing impairment?', 'Do you have a visual impairment?', 'Do you have non-de

In [28]:
def print_usage_summary():
    print("\n=== Token Usage Summary ===")
    for model, data in token_usage_summary.items():
        total_tokens = data["input_tokens"] + data["output_tokens"]
        print(f"{model}:")
        print(f"  Input tokens: {data['input_tokens']}")
        print(f"  Output tokens: {data['output_tokens']}")
        print(f"  Total tokens: {total_tokens}")
        print(f"  Total cost: ${data['cost']:.6f}")


In [32]:
# Step 1: Generate prompt
prompt = generate_demographic_mapping_prompt(unique_demographic_keys)

# Step 2: Collect responses (now includes GPT-4 Turbo)
mapping_responses = {
    "gpt-4":       get_cleaned_key_mapping(prompt, model_name="gpt-4"),
    "gpt-4-turbo": get_cleaned_key_mapping(prompt, model_name="gpt-4-turbo"),
    "gpt-3.5":     get_cleaned_key_mapping(prompt, model_name="gpt-3.5-turbo")
}

# Step 3: Preview responses
for model, response in mapping_responses.items():
    print(f"\n{model.upper()} Mapping Suggestion:\n")
    print(response)

# Step 4: Print usage and cost summary
print_usage_summary()


[gpt-4] Input tokens: 370, Output tokens: 458, Cost: $0.038580
[gpt-4-turbo] Input tokens: 370, Output tokens: 488, Cost: $0.018340
[gpt-3.5-turbo] Input tokens: 370, Output tokens: 441, Cost: $0.001252

GPT-4 Mapping Suggestion:

{
    'Age': 'Age',
    'Are you?': 'Gender',
    'Care for a family member /friend?': 'CaringForFamilyOrFriend',
    'Carer': 'Carer',
    'Disabled': 'DisabilityStatus',
    'Do you care for a family member/friend because of their illness or disability?': 'CaringForFamilyOrFriendDueToIllnessOrDisability',
    'Do you care for a family member/friend?': 'CaringForFamilyOrFriend',
    'Do you consider yourself disabled?': 'SelfIdentifiedDisability',
    'Do you consider yourself to be disabled?': 'SelfIdentifiedDisability',
    'Do you have a hearing impairment?': 'HearingImpairment',
    'Do you have a visual impairment?': 'VisualImpairment',
    'Do you have non-dependent children living with you?': 'NonDependentChildrenLivingAtHome',
    'Do you have pre-sc

In [33]:
# Step 2: Let user choose and extract mapping
demographic_key_mapping = choose_llm_mapping(mapping_responses)
normalize_all_demographics(unpacked_dfs, demographic_key_mapping)


 Available model responses:
1. GPT-4
2. GPT-4-TURBO
3. GPT-3.5

 Extracting mapping from GPT-4 response...

 Successfully extracted dictionary.
 Extracted Mapping:

  'Age': 'Age'
  'Are you?': 'Gender'
  'Care for a family member /friend?': 'CaringForFamilyOrFriend'
  'Carer': 'Carer'
  'Disabled': 'DisabilityStatus'
  'Do you care for a family member/friend because of their illness or disability?': 'CaringForFamilyOrFriendDueToIllnessOrDisability'
  'Do you care for a family member/friend?': 'CaringForFamilyOrFriend'
  'Do you consider yourself disabled?': 'SelfIdentifiedDisability'
  'Do you consider yourself to be disabled?': 'SelfIdentifiedDisability'
  'Do you have a hearing impairment?': 'HearingImpairment'
  'Do you have a visual impairment?': 'VisualImpairment'
  'Do you have non-dependent children living with you?': 'NonDependentChildrenLivingAtHome'
  'Do you have pre-school children living with you?': 'PreSchoolChildrenLivingAtHome'
  'Do you have pre-school children?': 'P

In [34]:
# 1. Extract keys
question_keys = collect_question_keys_from_unpacked_dfs(unpacked_dfs)

# 2. Get normalization from both models
mappings = normalize_question_keys_with_models(question_keys)



 Processed df_2023
 Processed df_2020
 Processed df_2016
 Processed df_2018
[gpt-4] input=647, output=841, cost=$0.069870

 GPT-4 Question Normalization Mapping:
 {'Active Travel': 'Active Travel', 'Care  Maintenance': 'Care & Maintenance', 'Care & Maintenance': 'Care & Maintenance', 'Are buildings and spaces well cared for?': 'Care & Maintenance', 'Care and maintenance': 'Care & Maintenance', 'Care__Maintenance_Are_buildings_and_spaces_well_cared_for?': 'Care & Maintenance', 'Facilities  Amenities': 'Facilities & Amenities', 'Facilities & Services': 'Facilities & Amenities', 'Facilities and amenities': 'Facilities & Amenities', 'Facilities__Amenities_Do_facilities_and_amenities_meet_my_needs?': 'Facilities & Amenities', 'Feeling Safe': 'Feeling Safe', 'Feeling safe': 'Feeling Safe', 'Housing  Community': 'Housing & Community', 'Housing & Community': 'Housing & Community', 'Housing and community': 'Housing & Community', 'Housing__Community_Does_housing_support_the_needs_of_the_communit

In [35]:
# 3. Let user choose which mapping to apply
selected_mapping = choose_question_key_mapping(mappings)

#  Apply selected mapping to all unpacked DataFrames
normalized_dfs = apply_mapping_to_unpacked_dfs(unpacked_dfs, selected_mapping)
unpacked_dfs = normalized_dfs  # Update canonical dict


 GPT-4 Normalization Mapping:

'Active Travel': 'Active Travel'
'Care  Maintenance': 'Care & Maintenance'
'Care & Maintenance': 'Care & Maintenance'
'Are buildings and spaces well cared for?': 'Care & Maintenance'
'Care and maintenance': 'Care & Maintenance'
'Care__Maintenance_Are_buildings_and_spaces_well_cared_for?': 'Care & Maintenance'
'Facilities  Amenities': 'Facilities & Amenities'
'Facilities & Services': 'Facilities & Amenities'
'Facilities and amenities': 'Facilities & Amenities'
'Facilities__Amenities_Do_facilities_and_amenities_meet_my_needs?': 'Facilities & Amenities'
'Feeling Safe': 'Feeling Safe'
'Feeling safe': 'Feeling Safe'
'Housing  Community': 'Housing & Community'
'Housing & Community': 'Housing & Community'
'Housing and community': 'Housing & Community'
'Housing__Community_Does_housing_support_the_needs_of_the_community_and_contribute_to_a_positive_environment?': 'Housing & Community'
'Identity & Belonging': 'Identity & Belonging'
'Identity and belonging': 'Ident

In [36]:


explode_questions_into_named_ratings(unpacked_dfs)
apply_macro_labels_from_namedquestion(unpacked_dfs, MACRO_THEME_MAP)
flattened_dfs = flatten_demographics_columns(unpacked_dfs)

unpacked_dfs = apply_type_inference_to_unpacked_dfs(unpacked_dfs)

unpacked_dfs = apply_type_inference_to_unpacked_dfs(flattened_dfs)

Exploded and updated: df_2023
Exploded and updated: df_2020
Exploded and updated: df_2016
Exploded and updated: df_2018
 Added macro_label from question_label to df_2023
 Added macro_label from question_label to df_2020
 Added macro_label from question_label to df_2016
 Added macro_label from question_label to df_2018
 Flattened demographics in df_2023 → stored as df_2023_flat
 Flattened demographics in df_2020 → stored as df_2020_flat
 Flattened demographics in df_2016 → stored as df_2016_flat
 Flattened demographics in df_2018 → stored as df_2018_flat
 Inferred dtypes for df_2023
 Inferred dtypes for df_2020
 Inferred dtypes for df_2016
 Inferred dtypes for df_2018
 Inferred dtypes for df_2023_flat
 Inferred dtypes for df_2020_flat
 Inferred dtypes for df_2016_flat
 Inferred dtypes for df_2018_flat


one shot + cot + instruction + gpt 4

In [50]:
import pandas as pd
from typing import TypedDict, Optional
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.callbacks import get_openai_callback
import os
import tiktoken

def count_tokens(text, model_name="gpt-4"):
    encoding = tiktoken.encoding_for_model(model_name)
    return len(encoding.encode(text))

#   Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-"

#   Replace with your real yearly DataFrames
df_2016 = df_2016_flat
df_2018 = df_2018_flat
df_2023 = df_2023_flat
df_2020 = df_2020_flat

class State(TypedDict):
    query: str
    code: Optional[str]
    result: Optional[pd.DataFrame]
    summary: Optional[str]
    error: Optional[str]         # ←  to capture execution errors
    feedback: Optional[str]      # ←  "correct" or "incorrect"

def extract_dataframe_info(df, year, questions_col='question_label', macro_col='macro_label'):
    columns_info = [(col, str(df[col].dtype)) for col in df.columns]

    # Collect unique question texts from string-based 'question_label' column
    question_label = []
    if questions_col in df.columns:
        question_label = sorted(
            df[questions_col].dropna().astype(str).unique().tolist()
        )

    # Collect unique macro_label values
    macro_values = []
    if macro_col in df.columns:
        macro_values = sorted(df[macro_col].dropna().unique().tolist())

    return {
        'year': year,
        'columns_info': columns_info,
        'question_label': question_label,
        'macro_values': macro_values,
    }

def llm_generate_code_node(state: State) -> State:
    query = state["query"]
    error = state.get("error", "")
    feedback = state.get("feedback", "")
    dfs_info = [
        extract_dataframe_info(df_2016, 2016, questions_col="question_label"),
        extract_dataframe_info(df_2018, 2018, questions_col="question_label"),
        extract_dataframe_info(df_2023, 2023, questions_col="question_label"),
        extract_dataframe_info(df_2020, 2020, questions_col="question_label"),
    ]

    # prompt_str = build_prompt(dfs_info)

    prompt_str = build_prompt(dfs_info, feedback=feedback, error=error)
    # print("\n FINAL PROMPT SENT TO GPT:\n")
    print(prompt_str)
    prompt_template = PromptTemplate(template=prompt_str, input_variables=["query"])
    llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-4", temperature=0.5), prompt=prompt_template)
    # model="gpt-4-turbo"
    # llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-4-turbo", temperature=0.5), prompt=prompt_template)
    # llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5), prompt=prompt_template)

    with get_openai_callback() as cb:
        raw_code = llm_chain.run({"query": query}).strip()
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f" Total Tokens: {cb.total_tokens}")
        print(f" Estimated Cost: ${cb.total_cost:.4f}")

    raw_code = llm_chain.run({"query": query}).strip()

    match = re.search(r"```python(.*?)```", raw_code, re.DOTALL)
    cleaned_code = match.group(1).strip() if match else raw_code.strip()

    return {
        "query": query,
        "code": cleaned_code,
        "result": None,
        "error": None,     # Clear previous error
        "feedback": None      # Clear previous feedback
    }

def execute_code_node(state: State) -> State:
    local_vars = {
        "df_2016": df_2016,
        "df_2018": df_2018,
        "df_2023": df_2023,
        "df_2020": df_2020,
        "pd": pd,
    }
    error_msg = None

    try:
        exec(state["code"], {}, local_vars)
        result = local_vars["result"] if "result" in local_vars else None

    except Exception as e:
        error_msg = f"{type(e).__name__}: {e}"  # capture exception type and message
        print("Error during execution:", error_msg)
        result = None  # or keep the error message in result if you want fallback behavior


    return {
        "query": state["query"],
        "code": state["code"],
        "result": result,
        "error": error_msg,
        "feedback": None
    }

def llm_summarize_node(state: State) -> State:
    result = state["result"]
    query = state["query"]

    # Convert result to string (smartly)
    if isinstance(result, pd.DataFrame):
        obs = result.to_markdown(index=False) if len(result) <= 10 else result.head().to_markdown(index=False)
    else:
        obs = str(result)

    # Generic summarization prompt
    prompt = PromptTemplate(
        template="""
You are a helpful analyst.

User Question:
{query}

Result:
{obs}

Instructions:
1. Interpret the result in the context of the user question.
Reason to understand the frame in which question is asked:
- Check the headers or labels in case of tabular data.
-  If it's a number or a single value, infer what it can mean for user query like a bolean value
- If it relates to polarity or sentiment, treat values > 0.1 as positive, < -0.1 as negative, and between as neutral.
- If it’s a string, extract the meaningful insight or decision from it.

Format:
Thought: Reason through what the result shows.
Answer: Provide a concise, human-readable summary that answers the question clearly.
""",
        input_variables=["query", "obs"],
    )

    chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0), prompt=prompt)
    # chain = LLMChain(llm=ChatOpenAI(model="gpt-4", temperature=0), prompt=prompt)


    summary = chain.run({"obs": obs, "query": query}).strip()

    return {
        "query":   query,
        "code":    state["code"],
        "result":  result,
        "summary": summary,
    }



# Construct LangGraph workflow
builder = StateGraph(State)
builder.add_node("generate_code", llm_generate_code_node)
builder.add_node("execute_code", execute_code_node)
builder.add_node("summarize", llm_summarize_node)
builder.set_entry_point("generate_code")
builder.add_edge("generate_code", "execute_code")
builder.add_edge("execute_code", "summarize")

#first one
# def build_prompt(dfs_info, query_placeholder="{query}"):
def build_prompt(dfs_info, query_placeholder="{query}", feedback: str = "", error: str = ""):
    prompt_blocks = []
    for info in dfs_info:
        year = info['year']
        columns_desc = "\n".join(f"- `{col}` ({dtype})" for col, dtype in info['columns_info'])
        # question_label_str = "\n".join(f"• {q}" for q in info.get("question_label", [])) or "[]"
        question_label_str = "[" + ", ".join(f'"{q}"' for q in info.get("question_label", [])) + "]"

        macro_str = str(info['macro_values']) if info['macro_values'] else "[]"

        block = f"""
  For `df_{year}`:
Columns:
{columns_desc}
- Each respondent has answered multiple survey questions so there are multiple rows for each 'id'.
The `question_label` column contains survey questions related to topics such as:
{question_label_str}

The `macro_label` column contains values:
{macro_str}
"""
        prompt_blocks.append(block.strip())

    full_schema = "\n\n".join(prompt_blocks)

    # Inject feedback dynamically here (this part was broken in your version)
    feedback_lines = []
    if feedback == "incorrect":
        # feedback_lines.append("The previous answer was marked incorrect by the user. Please revise accordingly.")
        feedback_lines.append(" ")
    if error:
        feedback_lines.append(f"The previous code failed with error: {error}")
    feedback_block = "\n".join(feedback_lines)

    prompt_template = f"""You are a Python assistant working with survey data across multiple years.

{full_schema}

Your task is as follows:
Rule one: Write the code that never fails.
Rule two: Never fail to follow rule one.
- Never try to aggregate comments
Reason on the below steps and create a plan and execute the code:
Step 1: Understand and Analyze the User Question
Read the user’s question carefully, which may be phrased naturally and informally.
Internally analyze which columns or fields of the DataFrame are relevant to answer this question.
Do not directly produce Python code or answers yet.
Use your reasoning to map the user’s intent to the DataFrame structure and data fields.
If the question is unclear, irrelevant, or unrelated to the DataFrame columns, respond with code to pri nt exactly:
"please clarify"
Identify if they are referring to any macro label or question label
Step 2: Identify Question Type
Determine if the question is pinpointed (fact-based) or detailed/explanatory based on wording cues:
If the question starts or contains words like "which", classify it as a pinpointed question expecting a concise answer.
If it contains phrases like "comment upon", "explain", "describe", or similar, classify it as a detailed/explanatory question expecting a fuller explanation. Use analytical function based upon this reasoning.

Step 3:

1. First, check whether the user's query matches or closely resembles any question in the `question_label` column. The user may input a partial string — treat it as a fuzzy match. If found, prioritize filtering or analyzing based on that. Use this value appropriately in coding.
2. Only fall back to `macro_label` if no relevant match is found in `question_label`.
3. Use semantic similarity, synonyms, and context clues to align query terms to survey questions.
4. Determine the time range requested: Is it a single year or a comparison across years and which dataframes are to be utilized?
5. Review the data types before writing code.
6. Handle nulls gracefully and avoid crashes from unhashable types.


When writing code:
- Always assign the final output to a variable called `result`.
- Always use the DataFrame variable `df1`.
- The `result` DataFrame must include (if present):
  - `sentiment_label`
  - `polarity`
  - Relevant demographics like age, region, gender
  -  Always reason while coding for counting tasks that weather grouping is required on the basis of  'id' or not.(even if user user does not mentions explicitly)
- Replace nulls in `comments` with empty strings.
- Even if no comments match, include the `comments` column with empty strings.
- Avoid duplicate entries.
- Use aggregate functions meaningfully.
- Check `result.empty` before accessing row values.

{feedback_block}
Query: {query_placeholder}

Return ONLY valid Python code (no markdown fences).
"""
    return prompt_template.strip()






In [51]:
import time
import sys

def run_query_with_retries(query, builder, max_retries=5, interactive_feedback=True):
    """
    Executes a query using the provided builder with automatic retries.

    Args:
        query (str): The natural language query to execute.
        builder: The LangGraph builder instance (already configured).
        max_retries (int): Maximum number of retry attempts.
        interactive_feedback (bool): Whether to ask for manual correction feedback.

    Returns:
        dict: Final output containing 'code', 'result', and 'summary'.
    """
    success_attempt = None
    final_output = None

    for attempt in range(1, max_retries + 1):
        print(f"\nAttempt {attempt} of {max_retries}")
        time.sleep(1.5)

        output = builder.compile().invoke({"query": query})
        result = output.get("result")
        error = output.get("error")

        if error:
            print(f" Execution failed with error: {error}")
            time.sleep(1)

            # Retry with feedback about the error
            output = builder.compile().invoke({
                "query": query,
                "feedback": "incorrect",
                "error": error
            })
            result = output.get("result")
            error = output.get("error")

            if error:
                print(f" Retry also failed with error: {error}")
                continue
            else:
                print(f" Recovered from failure on attempt {attempt}.")
                final_output = output
                success_attempt = attempt
                break
        else:
            print(f" Successfully executed query on attempt {attempt}. Result: {result}")
            final_output = output
            success_attempt = attempt
            break

    else:
        print(" All retries exhausted. Query failed.")
        return None

    # Display final output
    print(f"\n Final Output (attempt {success_attempt}):")
    print(" Generated Code:")
    print(final_output["code"])
    print("\nExecution Result:")
    print(final_output["result"])
    print("\nNLP Question:", query)
    print("\nSummary:")
    print(final_output["summary"])

    # Optional manual feedback loop
    if interactive_feedback:
        sys.stdout.flush()
        time.sleep(2)
        user_feedback = input("\nWas this answer correct? (y/n): ").strip().lower()
        if user_feedback == "n":
            print("\nRerunning with feedback that previous answer was incorrect...\n")
            output = builder.compile().invoke({
                "query": query,
                "feedback": "incorrect",
                "error": final_output.get("error", "")
            })

            print("\nRevised Code:")
            print(output["code"])
            print("\nRevised Execution Result:")
            print(output["result"])
            print("\nRevised Summary:")
            print(output["summary"])
            final_output = output

    return final_output


In [72]:
query ="How many males negatively think about parking in 2018 and 2023?"
final_result = run_query_with_retries(query, builder)

In [ ]:
# # Run the previous cells as well and create the databases as well
# Sample questions : Question Bank
# 1.	" How have gender-based negative opinions about traffic situation evolved between 2016 and 2023"
# 2.	"Has the proportion of respondents with caring responsibilities increased or decreased over time"
# 3.	“Explain the sentiment shift in stewardship from 2016 to 2023.”
# 4.	"How many people provided more than 4 rating to traffic in 2016?"
# 5.	"In which year did people rate the Movement feasibility the highest?"
# 6.	“Do people with mobility problems express lower sentiment in 2016?"
# 7.	"In which year did people rate the streets highest and natural space lowest?"
# 8.	"Do disabled respondents consistently give lower ratings?"
# 9.	"Is there a correlation between rating and sentiment polarity across years?"
# 10.	"What is the count of unique comments per macro_label in 2023?"
# 11.	"how many people re in the category of 55-64 age group in 2016?”
# 12.	“Explain the shift in the sentiment of people of who had children in terms of play and recreation”
# 13.	Which age saw the best overall sentiment change over the years?
# 14.	average rating of spaces in 2020 and also what was the mean of count of that category in 2018"
# 15.	“How many males negatively think about parking in 2018 and 2023?"


ingestion

In [101]:
# Example usage
start = time.time()
insert_all_flat_dataframes(
    unpacked_dfs,
    db_name="thematic_yearly_survey",
    user="postgres",
    password="PUT OWN PASSWORD",  # Replace with your actual password
    host="localhost",
    port="5432"
)
print(f" Done in {time.time() - start:.2f} seconds")



Inserting DataFrame into table: df_2016_flat_1
Inserting DataFrame into table: df_2018_flat_2
Inserting DataFrame into table: df_2020_flat_3
Inserting DataFrame into table: df_2023_flat_4
 All DataFrames inserted successfully!
 Done in 11.94 seconds


###### mongo ingestion

In [88]:
from pymongo import MongoClient
import pandas as pd

def clean_dataframe_for_mongo(df):
    """Convert NaNs and pandas.NA to None for MongoDB compatibility"""
    return df.where(pd.notna(df), None)

def insert_all_flat_dataframes_to_mongodb(unpacked_dfs, db_name="thematic_survey", host="localhost", port=27017):
    client = MongoClient(host=host, port=port)
    db = client[db_name]

    for name, df in unpacked_dfs.items():
        if name.startswith("df_") and name.endswith("_flat"):
            collection = db[name]  # Use the exact name of the DataFrame

            cleaned_df = clean_dataframe_for_mongo(df)
            documents = cleaned_df.to_dict(orient="records")

            if documents:
                collection.insert_many(documents)
                print(f" Inserted {len(documents)} documents into MongoDB collection '{name}'")

    client.close()

 # Ingest to MongoDB
start = time.time()
insert_all_flat_dataframes_to_mongodb(unpacked_dfs, db_name="thematic_survey")
print("Ingestion time", time.time() - start , "seconds")

 Inserted 25564 documents into MongoDB collection 'df_2016_flat'
 Inserted 20916 documents into MongoDB collection 'df_2018_flat'
 Inserted 67368 documents into MongoDB collection 'df_2020_flat'
 Inserted 35504 documents into MongoDB collection 'df_2023_flat'
Ingestion time 6.653691291809082 seconds
